In [13]:
 %pip install -q \
  "numpy>=1.26.4,<2.0.0" \
  "pandas" \
  "pyarrow" \
  "transformers" \
  "datasets" \
  "evaluate" \
  "accelerate"

In [1]:
# Cell 2 (after restarting runtime): Your original imports
import numpy as np, transformers, accelerate, datasets

print("numpy:", np.__version__)
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("datasets:", datasets.__version__)

numpy: 1.26.4
transformers: 4.45.1
accelerate: 0.32.1
datasets: 2.19.1


In [2]:
from accelerate.state import AcceleratorState
AcceleratorState._reset_state()

In [3]:
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    Trainer, TrainingArguments, DataCollatorForSeq2Seq
)
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import json
import os

In [4]:
from huggingface_hub import login
login()


In [5]:
data_files = {
    "train": "/content/dementia_train_multilang.json",
    "validation": "/content/dementia_validation_multilang.json",
    "test": "/content/dementia_test_multilang.json"
}

train_df = pd.read_json(data_files["train"])
validation_df = pd.read_json(data_files["validation"])
test_df = pd.read_json(data_files["test"])

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(validation_df),
    "test": Dataset.from_pandas(test_df)
})

In [6]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base", legacy=False)
tokenizer.clean_up_tokenization_spaces = True

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def ensure_response(ds):
    if "response" not in ds.column_names:
        ds = ds.add_column("response", [""] * len(ds))
    return ds

for split in dataset.keys():
    dataset[split] = ensure_response(dataset[split])

# --- 2) Preprocess (train/val use labels; test doesn't need them) -------
def preprocess_with_labels(batch):
    langs = batch.get("language", ["en"] * len(batch["input"]))
    inputs = [("émotion: " if l == "fr" else "emotion: ") + x for x, l in zip(batch["input"], langs)]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch["response"], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = targets["input_ids"]
    return model_inputs

def preprocess_no_labels(batch):
    langs = batch.get("language", ["en"] * len(batch["input"]))
    inputs = [("émotion: " if l == "fr" else "emotion: ") + x for x, l in zip(batch["input"], langs)]
    return tokenizer(inputs, padding="max_length", truncation=True, max_length=128)


In [8]:
def safe_remove(ds, cols, keep=None):
    keep = keep or []
    return [c for c in cols if (c in ds.column_names and c not in keep)]

cols_to_drop = ["input", "emotion", "intent", "tags",
                "care_mode", "language", "difficulty", "is_dementia_related", "response"]

tokenized = {}

In [9]:
tokenized["train"] = dataset["train"].map(
    preprocess_with_labels,
    batched=True,
    remove_columns=safe_remove(dataset["train"], cols_to_drop, keep=["response"])  # keep response!
)
tokenized["validation"] = dataset["validation"].map(
    preprocess_with_labels,
    batched=True,
    remove_columns=safe_remove(dataset["validation"], cols_to_drop, keep=["response"])
)
tokenized["test"] = dataset["test"].map(
    preprocess_no_labels,
    batched=True,
    remove_columns=safe_remove(dataset["test"], cols_to_drop, keep=["response"])
)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

In [10]:
for k in tokenized:
    tokenized[k].set_format("torch")

AcceleratorState._reset_state()

In [11]:
args = TrainingArguments(
    output_dir="./model",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",   # ✅ correct
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none",
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=data_collator, # Now this variable exists
    tokenizer=tokenizer
)

# Monkey patch the training_step to remove the optimizer.train() call
# This is a workaround for a compatibility issue between transformers and accelerate
original_training_step = trainer.training_step
def new_training_step(self, model, inputs):
    model.train()
    # Remove this line: if hasattr(self.optimizer, "train") and callable(self.optimizer.train):
    # Remove this line: self.optimizer.train()
    inputs = self._prepare_inputs(inputs)

    with self.accelerator.accumulate(model):
        tr_loss_step = self.compute_loss(model, inputs)

        if self.args.n_gpu > 1:
            tr_loss_step = tr_loss_step.mean()

        if self.args.gradient_accumulation_steps > 1 and not self.accelerator.undefine_state:
            # deepspeed, half precision, etc.
            self.accelerator.backward(tr_loss_step)
        else:
            tr_loss_step.backward()

        if self.accelerator.sync_gradients:
            self.accelerator.clip_grad_norm_(model.parameters(), self.args.max_grad_norm)

        self.optimizer.step()
        self.lr_scheduler.step()
        self.optimizer.zero_grad()

    return tr_loss_step.detach()

trainer.training_step = new_training_step.__get__(trainer, Trainer)


trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [ ]:
trainer.save_model("./model")
tokenizer.save_pretrained("./model")

model.push_to_hub("obx0x3/empathy-dementia")
tokenizer.push_to_hub("obx0x3/empathy-dementia")
print("✅ Model pushed to Hugging Face Hub.")

In [ ]:
from google.colab import files
import shutil

# Zip model directory
shutil.make_archive("empathy-dementia-model", 'zip', "./model")

# Download the zipped model
files.download("empathy-dementia-model.zip")
